In [ ]:
!pip install aiohttp pandas


In [ ]:
import pandas as pd
import aiohttp
import asyncio

# Replace 'YOUR_KAKAO_API_KEY' with your actual Kakao API key
KAKAO_API_KEY = 'YOUR_KAKAO_API_KEY'

In [ ]:

async def get_coordinates(session, address):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
    headers = {'Authorization': f'KakaoAK {KAKAO_API_KEY}'}
    async with session.get(url, headers=headers) as response:
        data = await response.json()
        if 'documents' in data and data['documents']:
            coordinates = data['documents'][0]['x'], data['documents'][0]['y']
            return coordinates
        else:
            return None

async def main():
    test_data = pd.read_excel('구청.xlsx')

    for i in range(len(test_data)):
        split = []
        if test_data['구청 주소'][i].endswith(']'):
            split = test_data['구청 주소'][i].split('[')
            test_data['구청 주소'][i] = split[0]

    async with aiohttp.ClientSession() as session:
        tasks = []
        missing_coordinates = []  # List to store addresses without coordinates
        for i in range(len(test_data)):
            address = test_data['구청 주소'][i]
            coordinates = await get_coordinates(session, address)  # Get coordinates using async function
            if coordinates:
                test_data.at[i, 'x_coordinate'] = coordinates[0]  # Update x_coordinate if coordinates are available
                test_data.at[i, 'y_coordinate'] = coordinates[1]  # Update y_coordinate if coordinates are available
            else:
                missing_coordinates.append(address)

    # Separate DataFrame for missing coordinates
    missing_coords_df = test_data[test_data['x_coordinate'].isnull()]

    # Save the DataFrame with coordinates to a CSV file
    test_data.to_csv('구청정리.csv', encoding='utf-8-sig', index=False)

    # Save the addresses without coordinates to a CSV file
    missing_coords_df.to_csv('missing_coordinates오피스텔전세.csv', encoding='utf-8-sig', index=False)

# Call the main function asynchronously
await main()
print(missing_coordinates)


Empty DataFrame
Columns: [시군구, 도로명, addr , x_coordinate, y_coordinate]
Index: []


In [ ]:
data.head()

,시군구,도로명,addr,x_coordinate,y_coordinate
0,서울특별시 강남구 개포동,개포로 264,서울특별시 강남구 개포동 개포로 264,127.052228,37.47975
1,서울특별시 강남구 개포동,개포로 264,서울특별시 강남구 개포동 개포로 264,127.052228,37.47975
2,서울특별시 강남구 개포동,개포로 264,서울특별시 강남구 개포동 개포로 264,127.052228,37.47975
3,서울특별시 강남구 개포동,개포로 264,서울특별시 강남구 개포동 개포로 264,127.052228,37.47975
4,서울특별시 강남구 개포동,개포로 264,서울특별시 강남구 개포동 개포로 264,127.052228,37.47975


In [ ]:
import pandas as pd

# 엑셀 파일 읽어오기
excel_file_path = "구청정리.csv"  # 엑셀 파일 경로와 파일명을 적절하게 수정해주세요

# 엑셀 파일을 DataFrame으로 읽어오기)
data = pd.read_csv(excel_file_path)

# 아파트 면적 컬럼 이름 (엑셀 파일 내에서 실제 컬럼명으로 수정해야 함)
area_column_name = "전용면적(㎡)"

# 평수 변환 함수
def square_meter_to_pyung(square_meter):
    pyung_conversion_factor = 1 / 3.3058
    pyung = square_meter * pyung_conversion_factor
    return pyung

# 평수로 변환하여 새로운 컬럼 추가
data["평수"] = data[area_column_name].apply(square_meter_to_pyung)

# 변환된 데이터를 엑셀 파일로 저장
output_excel_path = "구청정리끝.xlsx"  # 출력할 엑셀 파일 경로와 파일명을 지정해주세요
data.to_excel(output_excel_path, index=False)

print("평수 변환이 완료되었습니다.")
